In [0]:
%sql
DROP TABLE IF EXISTS flights;
DROP TABLE IF EXISTS flights_delta;

In [0]:
%sh

rm -r /dbfs/mnt/filesystem/flights_delta & rm -r /dbfs/mnt/filesystem/flights_parquet

In [0]:
df = spark.read.format('csv')\
  .option('header', True)\
  .option('inferSchema', True)\
  .load('dbfs:/databricks-datasets/flights/departuredelays.csv')\
  .sort('date')\
  .limit(2)

df.write.format('parquet')\
  .mode("overwrite")\
  .option('path', '/mnt/filesystem/flights_parquet')\
  .saveAsTable('flights')

In [0]:
%sql
SELECT * FROM flights

date,delay,distance,origin,destination
1010005,-8,2024,LAX,PBI
1010010,-6,1980,SEA,CLT


In [0]:
from pyspark.sql.functions import col

df = df.drop('delay')
df = df.withColumn('delay', col('origin'))
display(df)

date,distance,origin,destination,delay
1010005,2024,LAX,PBI,LAX
1010010,1980,SEA,CLT,SEA


In [0]:
df.write \
  .format('parquet') \
  .mode('append') \
  .save('/mnt/filesystem/flights_parquet')

In [0]:
%sql
REFRESH TABLE flights;
SELECT * FROM flights;

org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 4 times, most recent failure: Lost task 0.3 in stage 33.0 (TID 79) (10.139.64.5 executor 0): com.databricks.sql.io.FileReadException: Error while reading file dbfs:/mnt/filesystem/flights_parquet/part-00000-tid-7356201376228139896-0e7fee27-5ed9-4698-8f6c-f0092ecbb0c5-74-1-c000.snappy.parquet. Parquet column cannot be converted. Column: [delay], Expected: IntegerType, Found: BINARY
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.logFileNameAndThrow(FileScanRDD.scala:389)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:359)
	at org.apache.spark.util.NextIterator.hasNext(NextIterator.scala:73)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.$anonfun$prepareNextFile$1(FileScanRDD.scala:536)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.$anonfun$run$1(SparkThreadLocalForwardingThreadPoolExecutor.scala:104)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:68)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingHelper.runWithCaptured$(SparkThreadLocalForwardingThreadPoolExecutor.scala:54)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.runWithCaptured(SparkThreadLocalForwardingThreadPoolExecutor.scala:101)
	at org.apache.spark.util.threads.SparkThreadLocalCapturingRunnable.run(SparkThreadLocalForwardingThreadPoolExecutor.scala:104)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache.spark.sql.execution.datasources.SchemaColumnConvertNotSupportedException
	at com.databricks.sql.io.parquet.VectorizedColumnReader.constructConvertNotSupportedException(VectorizedColumnReader.java:448)
	at com.databricks.sql.io.parquet.VectorizedColumnReader.readBinaryBatch(VectorizedColumnReader.java:827)
	at com.databricks.sql.io.parquet.VectorizedColumnReader.readBatch(VectorizedColumnReader.java:406)
	at com.databricks.sql.io.parquet.DatabricksVectorizedParquetRecordReader.nextBatch(DatabricksVectorizedParquetRecordReader.java:480)
	at org.apache.spark.sql.execution.datasources.parquet.VectorizedParquetRecordReader.nextKeyValue(VectorizedParquetRecordReader.java:190)
	at org.apache.spark.sql.execution.datasources.RecordReaderIterator.hasNext(RecordReaderIterator.scala:38)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1$$anon$2.getNext(FileScanRDD.scala:333)
	... 18 more

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2766)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2713)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2707)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2707)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGScheduler.scala:1256)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1$adap

In [0]:
%sql
DESCRIBE flights

col_name,data_type,comment
date,int,null
delay,int,null
distance,int,null
origin,string,null
destination,string,null


In [0]:
%fs

ls /mnt/filesystem/flights_parquet

path,name,size
dbfs:/mnt/filesystem/flights_parquet/_SUCCESS,_SUCCESS,0
dbfs:/mnt/filesystem/flights_parquet/_committed_7356201376228139896,_committed_7356201376228139896,123
dbfs:/mnt/filesystem/flights_parquet/_committed_8949796578651985028,_committed_8949796578651985028,123
dbfs:/mnt/filesystem/flights_parquet/_started_7356201376228139896,_started_7356201376228139896,0
dbfs:/mnt/filesystem/flights_parquet/_started_8949796578651985028,_started_8949796578651985028,0
dbfs:/mnt/filesystem/flights_parquet/part-00000-tid-7356201376228139896-0e7fee27-5ed9-4698-8f6c-f0092ecbb0c5-74-1-c000.snappy.parquet,part-00000-tid-7356201376228139896-0e7fee27-5ed9-4698-8f6c-f0092ecbb0c5-74-1-c000.snappy.parquet,1288
dbfs:/mnt/filesystem/flights_parquet/part-00000-tid-8949796578651985028-d2f541cb-51af-4a7c-ba6e-7553063abffa-56-1-c000.snappy.parquet,part-00000-tid-8949796578651985028-d2f541cb-51af-4a7c-ba6e-7553063abffa-56-1-c000.snappy.parquet,1311


In [0]:
%sql
SELECT * FROM parquet.`/mnt/filesystem/flights_parquet/part-00000-tid-7356201376228139896-0e7fee27-5ed9-4698-8f6c-f0092ecbb0c5-74-1-c000.snappy.parquet`

date,distance,origin,destination,delay
1010005,2024,LAX,PBI,LAX
1010010,1980,SEA,CLT,SEA


In [0]:
%sql
SELECT * FROM parquet.`/mnt/filesystem/flights_parquet/part-00000-tid-6703560811808698918-3cd5f26c-d30b-45bf-a192-12c0c708432b-56-1-c000.snappy.parquet`

### DELTA

In [0]:
df = spark.read.format('csv')\
  .option('header', True)\
  .option('inferSchema', True)\
  .load('dbfs:/databricks-datasets/flights/departuredelays.csv')\
  .sort('date')\
  .limit(2)

df.write.format('delta')\
  .mode("overwrite")\
  .option('path', '/mnt/filesystem/flights_delta')\
  .saveAsTable('flights_delta')

In [0]:
%sql

SELECT * FROM flights_delta

date,delay,distance,origin,destination
1010005,-8,2024,LAX,PBI
1010010,-6,1980,SEA,CLT


#### Try to save some invalid schema to Delta

In [0]:
from pyspark.sql.functions import col

df = spark.read.format('csv')\
  .option('header', True)\
  .option('inferSchema', True)\
  .load('dbfs:/databricks-datasets/flights/departuredelays.csv')

df = df.drop('delay')
df = df.withColumn('delay', col('origin'))

df.write \
  .format('delta') \
  .mode('append') \
  .save('/mnt/filesystem/flights_delta')

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1646022116007039> in <module> 
 9 df = df . withColumn ( 'delay' , col ( 'origin' ) ) 
 10 
 ---> 11 df . write \ 
 12 . format ( 'delta' ) \ 
 13 . mode ( 'append' ) \ 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 1134 self . _jwrite . save ( ) 
 1135 else : 
 -> 1136 self . _jwrite . save ( path ) 
 1137 
 1138 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 114 # Hide where the exception came from that shows a non-Pythonic 
 115 # JVM exception message. 
 --> 116 raise converted from None 
 117 else : 
 118 raise 

 AnalysisException : Failed to merge fields 'delay' and 'delay'. Failed to merge incompatible data types IntegerType and StringType